In [1]:
# Importing required libraries

import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re

from gensim.models import Word2Vec
#from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

# nltk downloaded (run only once)
nltk.download('stopwords',quiet=True) # stopword library
nltk.download('wordnet', quiet=True) # wordnet library
nltk.download('words', quiet=True) # words library
nltk.download('punkt', quiet=True) # tokenize library


True

In [2]:
from langdetect import detect

1.A.

In [3]:
from zipfile import ZipFile
with ZipFile('blogs.zip', 'r') as z:
  print("Zip already extracted") if os.path.exists('blogs') else z.extractall(path='blogs')

Zip already extracted


In [4]:
df = pd.read_csv('blogs/blogtext.csv')
print(df.head())
print('-'*60)
df.info()

        id gender  age              topic      sign          date  \
0  2059027   male   15            Student       Leo   14,May,2004   
1  2059027   male   15            Student       Leo   13,May,2004   
2  2059027   male   15            Student       Leo   12,May,2004   
3  2059027   male   15            Student       Leo   12,May,2004   
4  3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   

                                                text  
0             Info has been found (+/- 100 pages,...  
1             These are the team members:   Drewe...  
2             In het kader van kernfusie op aarde...  
3                   testing!!!  testing!!!            
4               Thanks to Yahoo!'s Toolbar I can ...  
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id   

In [5]:
df['text'][0]

'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [6]:
df.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

- We can see that the data contains multiple columns for id, gender, age, topic, etc.. and text.
- age and id are of data type int64. others are object.
- text content the corpus of each blog.
- topic can be the classification as type of blog.

1.B.

i.

In [7]:
#checking null values
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

There are no null value in data

In [8]:
df_raw = df # creating a copy of raw data

In [9]:
# selecting only 5000 data for development

df = df[:5000]
df.shape

(5000, 7)

In [10]:
from tqdm import tqdm

In [11]:
for i in tqdm (range(len(df))):                                         
    try:                                                          
       lang = detect(df['text'][i])                                      
    except:                                                       
       df.drop(i, inplace=True)
    df['language'] = lang
df.reset_index(drop=True)

  0%|          | 0/5000 [00:00<?, ?it/s]C:\Users\ankit\AppData\Local\Temp\ipykernel_1568\3703312562.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = lang
 14%|█▍        | 710/5000 [00:03<00:17, 243.42it/s]C:\Users\ankit\AppData\Local\Temp\ipykernel_1568\3703312562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(i, inplace=True)
C:\Users\ankit\AppData\Local\Temp\ipykernel_1568\3703312562.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,id,gender,age,topic,sign,date,text,language
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",en
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,en
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,en
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,en
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,en
...,...,...,...,...,...,...,...,...
4983,1103575,female,17,indUnk,Scorpio,"27,August,2003",So... I had another one of those dreams...,en
4984,1103575,female,17,indUnk,Scorpio,"27,August,2003",mmm... strawberry tea for breakfast. To...,en
4985,1103575,female,17,indUnk,Scorpio,"27,August,2003","Yay for a new layout!! Yeah, I know, I...",en
4986,1103575,female,17,indUnk,Scorpio,"26,August,2003","Ok, so I lied... Fed up isn't playing F...",en


In [12]:
# saving df updated with langdetection to file using pickle

df.to_pickle("lang_df")

In [13]:
# loading df updated with langdetection from file using pickle

df = pd.read_pickle("lang_df")

In [14]:
df.head()

,id,gender,age,topic,sign,date,text,language
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",en
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,en
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,en
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,en
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,en


2.A., 2.B., 2.C., 2.D.

Preprocessing

In [15]:
## Preprocessing defenitions

def remove_punctuation(text):
    return re.sub('[^a-zA-Z]', ' ', str(text))

def lower_case(text):
    return text.lower()

def remove_tags(text):    
    return re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

def remove_special_chars_and_digits(text):
    return re.sub("(\\d|\\W)+"," ", text)

def remove_stop_words(text):
    tokenized_text = nltk.word_tokenize(text)
    text = [w for w in tokenized_text if not w in set(stopwords.words('english'))]
    return ' '.join(text)

def stopword_lemma(text):
    token = nltk.word_tokenize(text)
    text_stop = [x for x in token if x not in set(stopwords.words('english'))]
    lemmatizer = WordNetLemmatizer()
    text_lemma = [lemmatizer.lemmatize(word) for word in text_stop]
    text_lemma = ' '.join(text_lemma)
    return text_lemma

def remove_white_spaces(text):    
    text = text.strip()
    return text


def normalize_text(text: str) -> str:
    text = remove_punctuation(text)
    text = lower_case(text)
    text = remove_tags(text)
    text = remove_special_chars_and_digits(text)
    text = remove_stop_words(text)
    text = stopword_lemma(text)
    text = remove_white_spaces(text)
    
    return text

In [16]:
# Creating a new feature with normalized text

df['normalized_text'] = df['text'].apply(normalize_text)
df.loc[:, ['text', 'normalized_text']].head()

,text,normalized_text
0,"Info has been found (+/- 100 pages,...",info found page mb pdf file wait untill team l...
1,These are the team members: Drewe...,team member drewes van der laag urllink mail r...
2,In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,testing!!! testing!!!,testing testing
4,Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture url popups mean s...


3.A.

In [68]:
df_orig = df

In [69]:
df.dtypes

id                  int64
gender             object
age                 int64
topic              object
sign               object
date               object
text               object
language           object
normalized_text    object
dtype: object

In [72]:
data_new = df
data_new['age'] = df['age'].astype(str)
data_new['labels'] = df[['gender','age','topic','sign']].apply(lambda x:','.join(x), axis = 1) 
merged_data = data_new.drop(labels =['date','gender', 'age', 'topic', 'sign', 'id', 'language', 'text'], axis = 1)
merged_data.head()

,normalized_text,labels
0,info found page mb pdf file wait untill team l...,"male,15,Student,Leo"
1,team member drewes van der laag urllink mail r...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture url popups mean s...,"male,33,InvestmentBanking,Aquarius"


3.B.

In [76]:
X = merged_data['normalized_text']
y = merged_data['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 43)

In [77]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3341,), (1647,), (3341,), (1647,))

3.C.

Vectorize using CountVectorizer

In [79]:
vectorizer = CountVectorizer(min_df = 2, ngram_range = (1,2), stop_words = "english")

In [80]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [81]:
vectorizer_labels = CountVectorizer(min_df = 1, ngram_range = (1,1), stop_words = "english")

In [ ]:
labels = 

In [98]:
labels_vector = vectorizer_labels.fit_transform(labels)
label_classes = []
label_classes

[]

In [91]:
from sklearn.preprocessing import MultiLabelBinarizer

In [96]:
for  key in vectorizer_labels.vocabulary_.keys():
  label_classes.append(key)
mlb = MultiLabelBinarizer(classes = label_classes)

In [97]:
y = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in y]]
labels_trans = mlb.fit(labels) 
y_train = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in y_train]]
y_train = mlb.transform(y_train)
Y_test = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in y_test]]
y_test_trans = mlb.transform(y_test)

AttributeError: 'list' object has no attribute 'split'